In [123]:
import numpy as np
import itertools
import trimesh
import k3d
import matplotlib.pyplot as plt
%matplotlib inline

from trimesh.ray.ray_pyembree import RayMeshIntersector
from pypoisson import poisson_reconstruction

from igl import hausdorff
from pytorch3d.loss import chamfer_distance
from torch import FloatTensor

from utils.view import from_pose
# from utils.sampling import get_config

# !conda install -c conda-forge pyembree
# !conda install -c conda-forge igl
# !pip install Cython
# !pip install gym

In [124]:
def illustrate_points(points, plot=None, size=0.1):
    if plot is None:
        plot = k3d.plot(name='points')
    plt_points = k3d.points(positions=points, point_size=size)
    plot += plt_points
    plt_points.shader='3d'
    return plot

def illustrate_mesh(vertices, faces, plot=None):
    if plot is None:
        plot = k3d.plot()
        
    plt_surface = k3d.mesh(vertices, faces,
                           color_map = k3d.colormaps.basic_color_maps.Blues,
                           attribute=vertices[:, 2])

    plot += plt_surface
    return plot


def transform_mesh(mesh, rotation, translation=None, reverse=False):
    # rotation = from_pose([x, y, z], [0, 0, 0]) 
    mesh_ = mesh.copy()
    if reverse:
        if translation is not None:
            mesh_.apply_translation(-translation)
        mesh_.apply_transform(rotation.T)
    else:
        mesh_.apply_transform(rotation)
        if translation is not None:
            mesh_.apply_translation(translation)

    return mesh_


def transform_points(points, rotation, translation=None, reverse=False):
    if translation is None:
        translation = np.zeros((3))
    rotation = rotation[:3, :3]
    
    if reverse:
        return (points - translation).dot(rotation)
    return points.dot(rotation.T) + translation


def generate_sunflower_sphere_points(num_points=100):
    indices = np.arange(0, num_points, dtype=float) + 0.5

    phi = np.arccos(1 - 2 * indices / num_points)
    theta = np.pi * (1 + 5 ** 0.5) * indices

    x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)
    points = np.vstack([x, y, z]).T
    return points, phi, theta


## Sample Points

In [125]:
import torch
from utils.matrix_torch import (create_rotation_matrix_x, create_rotation_matrix_y, create_rotation_matrix_z, create_translation_matrix)


class ViewPoint:
    def __init__(self, point, phi, theta):
        self.point = point
        self.phi = phi
        self.theta = theta
        
    def get_transform_matrix(self):
        rotation = torch.mm(create_rotation_matrix_y(-self.phi),
                            create_rotation_matrix_z(-self.theta))
        
        translation = create_translation_matrix(0, 0, 0)
        transform = torch.mm(rotation, translation)
        return transform.t()
        

class Observation:
    def __init__(self, vertices, normals):
        self.vertices = vertices
        self.normals = normals
    
    def transform(self, transform):
        self.vertices = transform_points(self.vertices, transform)
        self.normals = transform_points(self.normals, transform,
                                        translation=None)
        
    def illustrate(self, plot=None):
        return illustrate_points(self.vertices, plot=plot)
    
    
class Model:
    def __init__(self, model_path):
        self.mesh = trimesh.load_mesh('./data/Dude.obj')
        self.transform = np.eye(4)
        
        self.view_points = []
    
    
    def generate_view_points(self, num_points=100):
        sphere_points, phis, thetas = generate_sunflower_sphere_points(num_points)

        dists = self.mesh.vertices - self.mesh.center_mass
        radius = np.abs(dists).max()
        radius *= 1.20

        sphere_points *= radius
        sphere_points += self.mesh.center_mass

        for point, phi, theta in zip(sphere_points, phis, thetas):
            self.view_points.append(ViewPoint(point, phi, theta))
            
            
    def get_point(self, view_point_idx):
        return self.view_points[view_point_idx].point
            

    def illustrate(self):
        plot = illustrate_mesh(self.mesh.vertices, self.mesh.faces)
        plot = illustrate_points([vp.point for vp in self.view_points],
                                 plot, size=0.5)
        return plot


    def rotate_to_view_point(self, view_point):
        self.transform = view_point.get_transform_matrix()
        self.mesh = transform_mesh(self.mesh, self.transform, reverse=True)
        
        
    def rotate_to_origin(self):
        self.mesh = transform_mesh(self.mesh, self.transform, reverse=False)
        self.transform = np.eye(4)

        
    def raycast(self, visibility_eps = 1e-6):
        vertex_ray_origins = np.array(self.mesh.vertices)
        vertex_depth = vertex_ray_origins[:, 2].copy()
        vertex_ray_origins[:, 2] = vertex_depth.max()
        vertex_ray_directions = np.tile(np.array([0, 0, -1]), (vertex_ray_origins.shape[0], 1))

        occluded = np.full_like(vertex_depth, True, dtype=np.bool)

        ray_mesh = RayMeshIntersector(self.mesh)

        triangles, rays, intersections = ray_mesh.intersects_id(
            ray_origins=vertex_ray_origins,
            ray_directions=vertex_ray_directions,
            multiple_hits=False,
            return_locations=True)

        occluded[rays] = intersections[:, 2] - vertex_depth[rays] > visibility_eps

        indices = np.where(~occluded)[0]
        vertices = self.mesh.vertices[indices]
        normals = self.mesh.vertex_normals[indices]

        return Observation(vertices, normals)


    def get_observation(self, view_point_idx):
        view_point = self.view_points[view_point_idx]
        self.rotate_to_view_point(view_point)
        
        observation = self.raycast()
        observation.transform(self.transform)
        
        self.rotate_to_origin()
        
        return observation
    
    
    def surface_similarity(self, reconstructed_vertices, reconstructed_faces):
        return hausdorff(self.mesh.vertices,
                         self.mesh.faces,
                         reconstructed_vertices,
                         reconstructed_faces.astype(np.int64))
        

    def observation_similarity(self, observation):
        gt = FloatTensor(np.expand_dims(self.mesh.vertices, axis=0))
        pred = FloatTensor(np.expand_dims(observation.vertices, axis=0))
        return chamfer_distance(gt, pred)

    
def get_mesh(observation):
    faces, vertices = poisson_reconstruction(
        observation.vertices, observation.normals, depth=10)
    return vertices, faces

def combine_observations(observations):
    # TODO unique
    combined_vertices = np.concatenate([ob.vertices for ob in observations])
    combined_normals = np.concatenate([ob.normals for ob in observations])
    return Observation(combined_vertices, combined_normals)


In [90]:
from time import sleep

NUM_POINTS = 10


plot = k3d.plot(name='points')
plot.display()

model = Model("./data/Dude.obj")
model.generate_view_points(NUM_POINTS)

observations = []
for view_point_idx in range(NUM_POINTS):
    observation = model.get_observation(view_point_idx)
    
    plot = observation.illustrate(plot)
    plot = illustrate_points([model.get_point(view_point_idx)], size=1.0, plot=plot)
    sleep(2)
    
    observations.append(observation)
    
combined_observation = EnvWrapper.combine_observations(observations)
reconstructed_vertices, reconstructed_faces = Environment.get_mesh(combined_observation)

loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)
print(loss)

illustrate_mesh(reconstructed_vertices, reconstructed_faces)

Output()

unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!


0.0780939793388279


/opt/conda/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

## Pipeline

In [238]:
import gym
from gym import spaces

MAX_POINS_CNT = 100000
VIEW_POINTS_CNT = 1000

class Environment(gym.Env):
    def __init__(self, number_of_view_points=VIEW_POINTS_CNT):
        super(Environment, self).__init__()

        self.number_of_view_points = number_of_view_points

        self.action_space = spaces.Discrete(number_of_view_points)
        self.observation_space = spaces.Box(
            -np.inf, np.inf, (MAX_POINS_CNT, 3), dtype=np.float32)

        self._similarity_threshold = 1e-10
        self._reconstruction_depth = 10

        self.model = None
        self.plot = None


    def reset(self):
        """
        Reset the environment for new episode.
        Randomly (or not) generate CAD model for this episode.
        """
        self.model = Model("./data/Dude.obj")
        self.model.generate_view_points(self.number_of_view_points)
        
        self.model.illustrate().display()
        
        init_state = self.action_space.sample()
        observation = self.model.get_observation(init_state)
        return observation


    def step(self, action):
        """
        Get new observation from current position (action), count step reward, decide whether to stop.
        Args:
            action: int
        return: 
            next_state: List[List[List[int, int, int]]]
            reward: float
            done: bool
            info: Tuple
        """
        observation = self.model.get_observation(action)

        reward = self.step_reward(observation)
        done = reward <= self._similarity_threshold

        return observation, reward, done, {}

    
    def render(self, action, observation, plot=None):
        if plot is None:
            plot = self.plot
            
        plot = illustrate_points(
           [self.model.get_point(action)], size=1.0, plot=plot)
        
        plot = observation.illustrate(plot)
        return plot
    
    
    def step_reward(self, observation):
        return float(self.model.observation_similarity(observation)[0])
    
    
    def final_reward(self, observation, illustrate=False):
        vertices, faces = self._get_mesh(observation)
        reward = self.model.surface_similarity(vertices, faces)
        
        if illustrate:
            illustrate_mesh(vertices, faces).display()
        return reward
        
        
    def _get_mesh(self, observation):
        faces, vertices = poisson_reconstruction(observation.vertices,
                                                 observation.normals,
                                                 depth=self._reconstruction_depth)
        return vertices, faces


    
class EnvWrapper:
    def __init__(self, environment):
        self.env = environment
        
        self.action_space = env.action_space
        self.observation_space = env.observation_space
        
        self._similarity_reward_weight = 1.0
        
        self.observations = []
        self.plot = None

    
    def reset(self):
        self.env.reset()
        self.observations = []
        
        self.plot = k3d.plot(name='wrapper')
        self.plot.display()

        
    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        self.observations.append(observation) # on GPU, paddings?
        
        combined_observation = self._combine_observations()
        reward = -1 + self._similarity_reward_weight * reward # compute here champher dist?
        
        return combined_observation, reward, done, info

    
    def render(self, action, observation):
        self.plot = self.env.render(action, observation, self.plot)
        
    
    def final_reward(self, illustrate=False):
        observation = self._combine_observations()
        return self.env.final_reward(observation, illustrate=illustrate)
    

    def _combine_observations(self):
        # TODO unique
        combined_vertices = np.concatenate([ob.vertices for ob in self.observations])
        combined_normals = np.concatenate([ob.normals for ob in self.observations])
        return Observation(combined_vertices, combined_normals)


In [241]:
class Agent:

    def __init__(self, env):
        self.env = env
        
        self.observation_size = env.observation_space.shape[0]
        self.actions_cnt = env.action_space.n
        
        self._max_iter = 10
        self._gamma = 0.99
        self._final_reward_weight = 1.0
  

    def predict_action(self, state):
        """
        Return action that should be done from input state according to current policy.
        Args:
            state: list of points - results of raycasting
        return: 
            action: int
        """    
        # some cool RL staff
        return self.env.action_space.sample()
    

    def evaluate(self):
        """
        Generate CAD model, reconstruct it and count the reward according   to MSE between original and reconstructed models and number of steps.
        Args:
            environment: Environment
            max_iter: int - max number of iterations to stop (~15)
            gamma: float - discounted factor
            w: float - weight of mse to final episode reward
        return: 
            episode_reward: float
        """    
        
        state = self.env.reset()
        
        episode_reward = 0.0
        for t in range(self._max_iter):
            action = self.predict_action(state)
            state, reward, done, info = self.env.step(action)
            self.env.render(action, state)
            episode_reward += reward * self._gamma ** t

            if done:
                break

        episode_reward += w * self.env.final_reward(illustrate=True)
        return episode_reward


In [242]:
env = Environment()
env_wrapper = EnvWrapper(env)

agent = Agent(env_wrapper)

In [243]:
agent.evaluate()

unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!


Output()

Output()

Output()

-3.7751655803162265